# Webscraper for transfermarkt

In [1]:
import time
import sys
from icecream import ic
from typing import List, Tuple, Dict, Set, Optional, Union

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException


import pandas as pd
from plotnine import *
import numpy as np

from loguru import logger

import os

%load_ext blackcellmagic

In [2]:
URL = "https://www.transfermarkt.de/premier-league/marktwerte/wettbewerb/GB1"
DRIVER_PATH = "/usr/bin/google-chrome"

In [3]:
options = Options()

# set up logger
logger.add("logging.log", mode="w")

1

In [8]:
def close_popup(driver):
    try:
        WebDriverWait(driver, 1).until(
            EC.frame_to_be_available_and_switch_to_it(
                (By.XPATH, '//*[@id="sp_message_iframe_953368"]')
            )
        )
        WebDriverWait(driver, 1).until(
            EC.element_to_be_clickable(
                (By.XPATH, '//*[@id="notice"]/div[3]/div[1]/div/button')
            )
        ).click()
        print("Cookies accepted")
    except Exception as e:
        print(e)
        print("No cookies button")

In [13]:
def setup_driver(timeout = 15):
    # Initialize the Chrome driver
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    driver.set_page_load_timeout(timeout)
    return driver

In [14]:
driver = setup_driver()
driver.get(URL)

wait = WebDriverWait(driver, 5)

close_popup(driver)

try:
    # store all player urls
    player_urls = []
    # get all player_names
    player_names = []

    # wait 5 seconds

    driver.implicitly_wait(5)
    link_elements = driver.find_elements_by_xpath(
        "//div[@id='yw1']/table//td[@class='hauptlink']/a"
    )
    market_values = [el.text for el in driver.find_elements_by_xpath(
        "//div[@id='yw1']/table//td[@class='rechts hauptlink']"
    )]
    
    for element in link_elements:
        href = element.get_attribute("href")
        text = (
            element.text
        )  # or use element.get_attribute('title') if the title is an attribute
        player_urls.append(href)
        player_names.append(text)
except:
    print("Could not fetch data")
finally:
    driver.quit()

Cookies accepted


In [25]:
# for each player url extract key data
players_data = []

for player_url in player_urls:
    driver = setup_driver()
    try:  
        driver.get(player_url)
        close_popup(driver)
    except:
        print("page took too long to load")
    driver.implicitly_wait(5)
    # get player age
    try:
        age = driver.find_element_by_xpath(
            '//li[contains(text(), "Alter")]'
        ).text
    except:
        age = None
    # get player height
    try:
        height = driver.find_element_by_xpath(
            '//li[contains(text(), "Größe")]'
        ).text
    except:
        height = None
    # get player position
    try:
        position = driver.find_element_by_xpath(
            '//li[contains(text(), "Position")]'
        ).text
    except:
        position = None
    # get player nationality
    try:
        nationality = driver.find_element_by_xpath(
            '//li[contains(text(), "Nationalität")]'
        ).text
    except:
        nationality = None
    try:
        # get player Berater
        berater = driver.find_element_by_xpath(
            '//li[contains(text(), "Berater")]'
        ).text
    except:
        berater = None



Cookies accepted
Größe: 1,95 m
Geb./Alter: 21.07.2000 (23)
Cookies accepted
Größe: 1,78 m
Geb./Alter: 05.09.2001 (22)
Cookies accepted
Größe: 1,71 m
Geb./Alter: 28.05.2000 (23)


KeyboardInterrupt: 

## Doing it for all Players in the Premier League

In [ ]:
URL_ALL = "https://www.transfermarkt.de/premier-league/startseite/wettbewerb/GB1"